In [ ]:
%%capture
!pip install transformers
!pip install -U sentence-transformers
!pip install datasets
!pip install evaluate
!pip install googletrans==3.1.0a0

from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from googletrans import Translator
from sentence_transformers import SentenceTransformer, util
import sys
import os
import pandas as pd
import numpy as np



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
tokenizer.pad_token = tokenizer.eos_token
if 'model' not in globals():
  model =  AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B", pad_token_id=tokenizer.eos_token_id)

In [ ]:
if 'emb_model' not in globals():
  emb_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')#, header=0, index_col = 0)

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Semantics In Intelligent Information Access/dataset_covered_recipes.csv",sep=';')#, header=0, index_col = 0)
risposta_utente = "oats,banana,milk"  #input("Hi! insert a recipe or ingredients: ") #meat
embedding_ing = emb_model.encode(risposta_utente)

EleutherAI/gpt-neo-2.7B text generation

In [ ]:
my_ingredients = 'oats,banana,milk'
embedding_ing = emb_model.encode(my_ingredients)

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

dataset_sustainable =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('HIGH'))]
dataset_sustainable.reset_index(inplace = True,drop = True)
embedding_recipes = emb_model.encode(dataset_sustainable['ingredients'])

# more sustainable
scores = []  
scores = util.pytorch_cos_sim(embedding_ing, embedding_recipes).flatten()
print(scores)
sorted = scores.argsort()[-3:]
print(scores[sorted])
high_results = dataset_sustainable.loc[sorted]

tensor([0.0759, 0.2650, 0.2650,  ..., 0.6454, 0.3984, 0.4800])
tensor([0.8304, 0.8824, 0.9423])


In [ ]:
dataset_mid_sustainable =  dataset.loc[np.where(dataset['SUSTAINABILITY'].eq('MEDIUM'))]
dataset_mid_sustainable.reset_index(inplace = True,drop = True)
embedding_mid_sus_recipes = emb_model.encode(dataset_mid_sustainable['ingredients'])

# not so sustainable
mid_sus_scores = []
mid_sus_scores = util.pytorch_cos_sim(embedding_ing, embedding_mid_sus_recipes).flatten()
print(mid_sus_scores)
mid_sorted = mid_sus_scores.argsort()[-3:]
print(mid_sus_scores[mid_sorted])
medium_results = dataset_mid_sustainable.loc[sorted]

tensor([0.6187, 0.4644, 0.6124,  ..., 0.4748, 0.4394, 0.5109])
tensor([0.7735, 0.7783, 0.7811])


## High sustainable

In [ ]:
from transformers import set_seed

set_seed(42)

In [ ]:
title = high_results.iloc[2,1]
url = high_results.iloc[2, 2]
ingredients = high_results.iloc[2, 3]

display(high_results, title, url, ingredients)

,recipe_id,title,url,ingredients,SUSTAINABILITY
487,35477,Survival Necklace,http://www.food.com/recipe/survival-necklace-9...,"banana,oat meal",HIGH
1951,50516,Strawberry Banana Smoothie,http://www.kraftrecipes.com/recipes/strawberry...,"cow milk,banana,strawberry,oat meal",HIGH
1728,2818,Oatmeal and Bananas Breakfast (Baby Food),http://www.food.com/recipe/oatmeal-and-bananas...,"oat,cow milk,banana",HIGH


'Oatmeal and Bananas Breakfast (Baby Food)'

'http://www.food.com/recipe/oatmeal-and-bananas-breakfast-baby-food-52539'

'oat,cow milk,banana'

In [ ]:
prompt = ['A recipe '+ingredients]

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text1 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text1[0]

[{'generated_text': 'A recipe oat,cow milk,banana cream and pineapple juice with sweetened shredded coconut and fresh pineapple,tossed with a little grated ginger in a bowl made into a mold and'}]

In [ ]:
prompt = ['What do you think about '+title+' it has high sustainability']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text2 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text2[0]

[{'generated_text': 'What do you think about Oatmeal and Bananas Breakfast (Baby Food) it has high sustainability.\n\nDo you like the idea of Organic Bananas, Oatmeal (Breakfast Cereal)'}]

In [ ]:
generated_text_high_sustainable = str(generated_text1[0][0]['generated_text']) + '.' + str(generated_text2[0][0]['generated_text']) + ', you can find more on: ' + high_results.iloc[2, 2]
print(generated_text_high_sustainable.replace('\n', ''))

A recipe oat,cow milk,banana cream and pineapple juice with sweetened shredded coconut and fresh pineapple,tossed with a little grated ginger in a bowl made into a mold and.What do you think about Oatmeal and Bananas Breakfast (Baby Food) it has high sustainability.Do you like the idea of Organic Bananas, Oatmeal (Breakfast Cereal), you can find more on: http://www.food.com/recipe/oatmeal-and-bananas-breakfast-baby-food-52539


## Medium sustainable

In [ ]:
title = medium_results.iloc[2,1]
url = medium_results.iloc[2, 2]
ingredients = medium_results.iloc[2, 3]

display(medium_results, title, url, ingredients)

,recipe_id,title,url,ingredients,SUSTAINABILITY
487,1305,Grapefruit Oil,http://cooking.nytimes.com/recipes/3950,"canola oil,grapefruit juice",MEDIUM
1951,42439,Milk Gravy (Paula Deen),http://www.food.com/recipe/milk-gravy-paula-de...,"olive oil,cow milk,wheat plain flour,black pep...",MEDIUM
1728,1100,Pressed Pie Crust,http://tastykitchen.com/recipes/desserts/press...,"olive oil,cow milk,wheat plain flour,cane sugar",MEDIUM


'Pressed Pie Crust'

'http://tastykitchen.com/recipes/desserts/pressed-pie-crust/'

'olive oil,cow milk,wheat plain flour,cane sugar'

In [ ]:
prompt = ['A recipe '+ingredients]

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text1 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text1[0]

[{'generated_text': "A recipe olive oil,cow milk,wheat plain flour,cane sugar,honey,baker's yeast,water,eggs,water,lemon juice,parsley,"}]

In [ ]:
prompt = ['What do you think about '+title+'? it has medium sustainability']

text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, output_scores=True)
generated_text2 = text_generation_pipeline(prompt, max_length=40, do_sample=True)

generated_text2[0]

[{'generated_text': 'What do you think about Pressed Pie Crust? it has medium sustainability at 1.5% CO2. There is also a very long life and no need for baking for several baking cycles.'}]

In [ ]:
generated_test_medium_sustainable = generated_text_high_sustainable = str(generated_text1[0][0]['generated_text']) + '.' + str(generated_text2[0][0]['generated_text']) + ', you can find more on: ' + high_results.iloc[2, 2]
print(generated_text_high_sustainable.replace('\n', ''))

A recipe olive oil,cow milk,wheat plain flour,cane sugar,honey,baker's yeast,water,eggs,water,lemon juice,parsley,.What do you think about Pressed Pie Crust? it has medium sustainability at 1.5% CO2. There is also a very long life and no need for baking for several baking cycles., you can find more on: http://www.food.com/recipe/oatmeal-and-bananas-breakfast-baby-food-52539
